In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/gtlim/Sample_Submit.csv
/kaggle/input/gtlim/train_img/437/45005_30_s_3.jpg
/kaggle/input/gtlim/train_img/437/45005_60_s_21.jpg
/kaggle/input/gtlim/train_img/437/45005_00_s_1.jpg
/kaggle/input/gtlim/train_img/515/50162_30_s_4.jpg
/kaggle/input/gtlim/train_img/515/50162_30_s_6.jpg
/kaggle/input/gtlim/train_img/515/50162_30_s_1.jpg
/kaggle/input/gtlim/train_img/248/25419_00_s_5.jpg
/kaggle/input/gtlim/train_img/248/25419_30_s_23.jpg
/kaggle/input/gtlim/train_img/248/25419_00_s_3.jpg
/kaggle/input/gtlim/train_img/625/60270_30_s_11.jpg
/kaggle/input/gtlim/train_img/625/60270_00_s_8.jpg
/kaggle/input/gtlim/train_img/625/60270_30_s_24.jpg
/kaggle/input/gtlim/train_img/7/10096_60_s_10.jpg
/kaggle/input/gtlim/train_img/7/10096_30_s_5.jpg
/kaggle/input/gtlim/train_img/7/10096_0_s_1.jpg
/kaggle/input/gtlim/train_img/843/80266_00_s_5.jpg
/kaggle/input/gtlim/train_img/843/80266_30_s_5.jpg
/kaggle/input/gtlim/train_img/843/80266_60_s_12.jpg
/kaggle/input/gtlim/train_img/135/15613_30_s_

In [2]:
import torch
import random
import tqdm
import cv2
import os
import copy
import torchvision
import PIL

torch.manual_seed(1)
random.seed(1)

submit = pd.read_csv('../input/gtlim/Sample_Submit.csv')

In [3]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self,split,path,transform):
        super(Dataset,self).__init__()
        self.split = split
        self.root_path = path
        self.transform = transform
        self.video_list = list()
        if split == 'train':
            self.labels = list()
            for cls in os.listdir(path):
                self.video_list.extend(os.listdir(os.path.join(path,cls)))
                self.labels.extend(np.full((len(os.listdir(os.path.join(path,cls)))),int(cls)))
        elif split == 'test':
            self.video_list = os.listdir(path)
            
    def __len__(self):
        return len(self.video_list)
    
    def __getitem__(self,index):
        if self.split == 'train':
            img = PIL.Image.open(os.path.join(self.root_path,str(self.labels[index]),self.video_list[index]))
            img = img.convert('RGB')
            img = self.transform(img)
            return img,self.labels[index]
        
        elif self.split == 'test':
            img = PIL.Image.open(os.path.join(self.root_path,str(index)+'.jpg'))
            img = img.convert('RGB')
            img = self.transform(img)
            return img

In [4]:
import torchvision.transforms as transforms

torch.manual_seed(1)
random.seed(1)

transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Resize(224),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])


train_dataset = Dataset(path='../input/gtlim/train_img',
                        split = 'train',
                        transform = transform)


test_dataset = Dataset(path='../input/gtlim/test_img',
                       split = 'test',
                       transform = transform)


train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=100,
                                           shuffle=True,
                                           num_workers=2,
                                           drop_last = True
                                          )


test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=100,
                                          shuffle=False,
                                          num_workers=0,
                                         )

In [5]:
torch.manual_seed(1)
random.seed(1)

model = torchvision.models.resnet18(pretrained=True)

model.layer1.requires_grad=False
model.layer2.requires_grad=False
model.layer3.requires_grad=False
model.layer4.requires_grad=False

def resnetplus(model):
    linear1 = torch.nn.Linear(1000, 512)
    linear2 = torch.nn.Linear(512, 256)
    linear3 = torch.nn.Linear(256, 900)

    torch.nn.init.xavier_normal_(linear1.weight)
    torch.nn.init.xavier_normal_(linear2.weight)
    torch.nn.init.xavier_normal_(linear3.weight)

    relu = torch.nn.LeakyReLU()
    dropout = torch.nn.Dropout(0.2)
    res_model = torch.nn.Sequential(model, 
                                    linear1, relu, dropout,
                                    linear2, relu, dropout,
                                    linear3)
    return res_model
    
resnetplus(model)


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

Sequential(
  (0): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_runnin

In [6]:

model = model.cuda()
model.train()

Loss = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(),lr=0.1,momentum=0.9,weight_decay=0.00005)

Total_Epoch = 30

for epoch in range(Total_Epoch):
    epoch_loss = 0
    epoch_accuracy = 0
    for X,Y in tqdm.tqdm(train_loader):
        X = torch.FloatTensor(X).cuda()
        Y = torch.LongTensor(Y).cuda()

        output = model(X)
        Cost = Loss(output,Y)

        optimizer.zero_grad()
        Cost.backward()
        optimizer.step()

        hypothesis = torch.nn.functional.softmax(output,dim=1)
        predict = torch.argmax(hypothesis,dim=1)
        correct_predict = predict == Y

        epoch_accuracy = epoch_accuracy + correct_predict.sum()
        epoch_loss = epoch_loss + Cost.item()

    if epoch % 1 == 0:
        print(" Epoch : {:4d} , Cost : {:.2f} , Accuracy : {:.4f}".format(epoch,epoch_loss,epoch_accuracy/len(train_loader.dataset)))
        

100%|██████████| 27/27 [00:17<00:00,  1.54it/s]


 Epoch :    0 , Cost : 202.45 , Accuracy : 0.0004


100%|██████████| 27/27 [00:07<00:00,  3.81it/s]


 Epoch :    1 , Cost : 192.04 , Accuracy : 0.0011


100%|██████████| 27/27 [00:07<00:00,  3.73it/s]


...

 Epoch :   27 , Cost : 1.08 , Accuracy : 0.9933


100%|██████████| 27/27 [00:07<00:00,  3.80it/s]


 Epoch :   28 , Cost : 0.88 , Accuracy : 0.9952


100%|██████████| 27/27 [00:06<00:00,  3.98it/s]

 Epoch :   29 , Cost : 0.61 , Accuracy : 0.9963


In [7]:
total_prediction = list()
model.eval()
with torch.no_grad():
    for X in tqdm.tqdm(test_loader):
        X = torch.FloatTensor(X).cuda()
        output = model(X)
        hypothesis = torch.nn.functional.softmax(output,dim=1)
        predict = torch.argmax(hypothesis,dim=1)
        total_prediction.extend(predict.cpu().numpy())
    submit['label'] = total_prediction
    submit.to_csv('submit.csv',index=False)

100%|██████████| 18/18 [00:16<00:00,  1.12it/s]
